In [1]:
from google.colab import drive
import os
import json
import numpy as np
from PIL import Image
from tqdm import tqdm

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import json
import cv2
import re
import numpy as np

# Paths to videos and labels
videos_path = "/content/drive/MyDrive/NUS_ISS_Talent_Experience_Resumes/cholect50-challenge-val/videos"
labels_path = "/content/drive/MyDrive/NUS_ISS_Talent_Experience_Resumes/cholect50-challenge-val/labels"

# Load all target names from the first JSON (dynamic)
example_json = next(f for f in os.listdir(labels_path) if f.endswith('.json'))
with open(os.path.join(labels_path, example_json), 'r') as f:
    data = json.load(f)
# Build mapping: target_id -> target_name
target_categories = data['categories']['target']
# Correct dynamic target mapping
target_id_to_name = {int(k): v for k, v in data['categories']['target'].items()}
num_targets = len(target_id_to_name)


# Load triplet -> target mapping from label_mapping.txt
target_mapping = {}
with open(os.path.join('/content/drive/MyDrive/NUS_ISS_Talent_Experience_Resumes/cholect50-challenge-val/label_mapping.txt'), 'r') as f:
    for line in f:
        line = line.strip()
        if not line or line.startswith('#'):
            continue
        parts = line.split(',')
        triplet_id = int(parts[0])
        target_id = int(parts[3])
        target_mapping[triplet_id] = target_id

X = []
Y = []

# Loop through all video folders
for vid_folder in sorted(os.listdir(videos_path)):
    vid_path = os.path.join(videos_path, vid_folder)
    if not os.path.isdir(vid_path):
        continue

    label_file = f"{vid_folder}.json"
    label_path = os.path.join(labels_path, label_file)
    if not os.path.exists(label_path):
        print(f"Warning: {label_path} not found, skipping {vid_folder}")
        continue

    with open(label_path, 'r') as f:
        data = json.load(f)
    annotations = data['annotations']

    # Build frame_number -> list of target IDs
    frame_target = {}
    for frame_id, triplets in annotations.items():
        frame_number = int(frame_id)
        targets_in_frame = []

        for triplet in triplets:
            triplet_id = triplet[0]
            target_id = target_mapping.get(triplet_id, -1)
            if target_id != -1:
                targets_in_frame.append(target_id)

        if len(targets_in_frame) == 0:
            continue  # skip frames with no valid target

        frame_target[frame_number] = targets_in_frame

    # Process frames
    frame_files = sorted([f for f in os.listdir(vid_path) if f.endswith('.png')])
    for frame_file in frame_files:
        match = re.match(r"(\d+)", frame_file)
        if not match:
            continue
        frame_number = int(match.group(1))
        targets = frame_target.get(frame_number)
        if not targets:
            continue

        # Create multi-hot vector
        multi_hot = np.zeros(num_targets, dtype=np.int64)
        for target_id in targets:
            multi_hot[target_id] = 1

        # Read image
        img_path = os.path.join(vid_path, frame_file)
        img = cv2.imread(img_path)
        if img is None:
            continue

        img = cv2.resize(img, (224, 224))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        X.append(img)
        Y.append(multi_hot)

# Convert to numpy arrays
X = np.array(X, dtype=np.float32) / 255.0
Y = np.array(Y, dtype=np.int64)

print(f"Total frames processed: {len(X)}")
print(f"X shape: {X.shape}, Y shape: {Y.shape}")
print(f"Example multi-hot target vectors:\n{Y[:10]}")
print(f"Target ID -> Name mapping:\n{target_id_to_name}")

Total frames processed: 1209
X shape: (1209, 224, 224, 3), Y shape: (1209, 15)
Example multi-hot target vectors:
[[1 0 0 0 0 0 0 0 1 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
Target ID -> Name mapping:
{0: 'gallbladder', 1: 'cystic_plate', 2: 'cystic_duct', 3: 'cystic_artery', 4: 'cystic_pedicle', 5: 'blood_vessel', 6: 'fluid', 7: 'abdominal_wall_cavity', 8: 'liver', 9: 'adhesion', 10: 'omentum', 11: 'peritoneum', 12: 'gut', 13: 'specimen_bag', 14: 'null_target'}


In [3]:
import numpy as np

# Y_train is assumed to be a numpy array of shape (num_frames, num_classes)
class_sums = np.sum(Y, axis=0)
for i, s in enumerate(class_sums):
    print(f"Class {i}: {s} positive samples")

Class 0: 360 positive samples
Class 1: 56 positive samples
Class 2: 143 positive samples
Class 3: 140 positive samples
Class 4: 42 positive samples
Class 5: 41 positive samples
Class 6: 120 positive samples
Class 7: 56 positive samples
Class 8: 124 positive samples
Class 9: 0 positive samples
Class 10: 36 positive samples
Class 11: 0 positive samples
Class 12: 3 positive samples
Class 13: 132 positive samples
Class 14: 100 positive samples


In [4]:
import numpy as np

# Y is your multi-hot label array of shape (num_samples, num_classes)
class_counts = np.sum(Y, axis=0)  # sum over samples for each class
valid_classes = np.where(class_counts > 0)[0]

print("Classes with at least one frame:", valid_classes)
print("Number of valid classes:", len(valid_classes))

Classes with at least one frame: [ 0  1  2  3  4  5  6  7  8 10 12 13 14]
Number of valid classes: 13


In [31]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# --- Function to augment a single frame ---
def augment_frame(img):
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_flip_up_down(img)
    #k = tf.random.uniform([], minval=0, maxval=4, dtype=tf.int32)
    #img = tf.image.rot90(img, k)
    img = tf.image.random_brightness(img, max_delta=0.1)
    # img = tf.image.random_contrast(img, lower=0.9, upper=1.1)
    return img

# --- Identify classes with at least 1 positive ---
class_counts = np.sum(Y, axis=0)
valid_classes = np.where(class_counts > 0)[0]

# --- Target samples per class ---
target_samples = 200

aug_X = []
aug_Y = []

for cls in valid_classes:
    current_count = np.sum(Y[:, cls])
    if current_count >= target_samples:
        continue  # skip majority classes
    needed = target_samples - current_count
    idx = np.where(Y[:, cls] == 1)[0]
    if len(idx) == 0:
        continue  # skip if no samples
    np.random.shuffle(idx)
    for i in range(needed):
        img = X[idx[i % len(idx)]]  # loop over available frames
        label = Y[idx[i % len(idx)]]
        aug_img = augment_frame(img)
        aug_X.append(aug_img.numpy())
        aug_Y.append(label)

# Combine augmented data with original
X_balanced = np.concatenate([X, np.array(aug_X)], axis=0)
Y_balanced = np.concatenate([Y, np.array(aug_Y)], axis=0)

print("Original dataset size:", len(X))
print("Balanced dataset size:", len(X_balanced))

Original dataset size: 1209
Balanced dataset size: 2616


In [6]:
import numpy as np

# X is your numpy array of shape (N, H, W, 3), values in 0..1 (if normalized)
# If X is 0..255, remove the /255.0 part

X_min = np.min(X_balanced)
X_max = np.max(X_balanced)
X_mean = np.mean(X_balanced)

print(f"✅ Pixel value stats for X:")
print(f"Min pixel value: {X_min}")
print(f"Max pixel value: {X_max}")
print(f"Mean pixel value: {X_mean:.4f}")

# Optionally, compute per-channel stats
X_mean_channels = np.mean(X_balanced, axis=(0,1,2))
X_min_channels = np.min(X_balanced, axis=(0,1,2))
X_max_channels = np.max(X_balanced, axis=(0,1,2))

print(f"\nPer-channel mean: {X_mean_channels}")
print(f"Per-channel min: {X_min_channels}")
print(f"Per-channel max: {X_max_channels}")

✅ Pixel value stats for X:
Min pixel value: -0.09999366104602814
Max pixel value: 1.0997942686080933
Mean pixel value: 0.2534

Per-channel mean: [0.13073638 0.12797154 0.11789288]
Per-channel min: [-0.09956763 -0.09999366 -0.09999366]
Per-channel max: [1.0997943 1.0997943 1.0997586]


In [32]:
import numpy as np

# Count positives per class
target_counts = Y_balanced.sum(axis=0)
print("Target counts per class:", target_counts)

# Find valid (non-empty) classes
valid_mask = target_counts > 0
valid_indices = np.where(valid_mask)[0]

# Filter Y_balanced to keep only valid classes
Y_balanced_filtered = Y_balanced[:, valid_mask]

print(f"✅ Removed {np.sum(~valid_mask)} empty classes")
print(f"New Y_balanced shape: {Y_balanced_filtered.shape}")
print("Remaining class indices:", valid_indices)

Target counts per class: [589 200 200 200 200 200 202 201 220   0 201   0 200 212 213]
✅ Removed 2 empty classes
New Y_balanced shape: (2616, 13)
Remaining class indices: [ 0  1  2  3  4  5  6  7  8 10 12 13 14]


In [33]:
import numpy as np

# Y_balanced now only has valid classes
target_counts = Y_balanced_filtered.sum(axis=0)  # number of positive samples per class
total_frames = Y_balanced_filtered.shape[0]
num_classes = Y_balanced_filtered.shape[1]

# Multi-label class weight formula: inverse frequency
class_weights = total_frames / (num_classes * target_counts)

# Optional: normalize to have mean=1
class_weights = class_weights / np.mean(class_weights)

print("✅ Class weights:")
for i, w in enumerate(class_weights):
    print(f"Class {i}: {w:.3f}")

✅ Class weights:
Class 0: 0.364
Class 1: 1.073
Class 2: 1.073
Class 3: 1.073
Class 4: 1.073
Class 5: 1.073
Class 6: 1.063
Class 7: 1.068
Class 8: 0.976
Class 9: 1.068
Class 10: 1.073
Class 11: 1.013
Class 12: 1.008


In [34]:
# --- Split into train/val/test ---
X_train, X_temp, Y_train, Y_temp = train_test_split(
    X_balanced, Y_balanced_filtered, test_size=0.3, random_state=42, shuffle=True
)
X_val, X_test, Y_val, Y_test = train_test_split(
    X_temp, Y_temp, test_size=0.5, random_state=42, shuffle=True
)

print("Train size:", len(X_train))
print("Val size:", len(X_val))
print("Test size:", len(X_test))

Train size: 1831
Val size: 392
Test size: 393


In [35]:
import tensorflow as tf

BATCH_SIZE = 16
AUTOTUNE = tf.data.AUTOTUNE

def make_dataset(X, Y, batch_size=BATCH_SIZE, training=False):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    if training:
        dataset = dataset.shuffle(2048)
    dataset = dataset.batch(batch_size).prefetch(AUTOTUNE)
    return dataset

train_ds = make_dataset(X_train, Y_train, training=True)
val_ds   = make_dataset(X_val, Y_val, training=False)
test_ds  = make_dataset(X_test, Y_test, training=False)

In [36]:
import tensorflow as tf

def focal_loss_with_class_weights(class_weights, gamma=2.0, alpha=0.25):
    class_weights = tf.constant(class_weights, dtype=tf.float32)

    def loss(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)
        bce = -(y_true * tf.math.log(y_pred) + (1 - y_true) * tf.math.log(1 - y_pred))
        fl = alpha * tf.pow(1 - y_pred, gamma) * y_true * bce + \
             (1 - alpha) * tf.pow(y_pred, gamma) * (1 - y_true) * bce
        # apply per-class weights
        weighted_fl = fl * class_weights
        return tf.reduce_mean(weighted_fl)
    return loss

In [37]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2

IMG_SIZE = 224
num_targets = Y_balanced_filtered.shape[1]
drop_rate = 0.3
num_heads = 4  # multihead attention heads
embed_dim = 128  # feature dimension for attention

def build_fast_mha_model(input_shape=(IMG_SIZE, IMG_SIZE, 3), num_classes=num_targets):
    # === Backbone ===
    base = MobileNetV2(include_top=False, weights='imagenet', input_shape=input_shape, pooling=None)
    base.trainable = False  # freeze backbone for speed

    inp = layers.Input(shape=input_shape)
    x = base(inp)  # shape: (batch, h/32, w/32, channels)

    # Flatten spatial dimensions for attention: (batch, seq_len, channels)
    b, h, w, c = x.shape
    x_flat = layers.Reshape((-1, c))(x)  # seq_len = h*w

    # === Multi-Head Attention ===
    attn_out = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(x_flat, x_flat)
    attn_out = layers.GlobalAveragePooling1D()(attn_out)

    # === Dense Head ===
    x = layers.Dropout(drop_rate)(attn_out)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(drop_rate)(x)
    out = layers.Dense(num_classes, activation='sigmoid')(x)  # multi-label output

    model = models.Model(inputs=inp, outputs=out)
    return model

# Build model
model = build_fast_mha_model()
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mobilenetv2_1.00_2… │ (None, 7, 7,      │  2,257,984 │ input_layer_7[0]… │
│ (Functional)        │ 1280)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_3 (Reshape) │ (None, 49, 1280)  │          0 │ mobilenetv2_1.00… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 49, 1280)  │  2,624,256 │ reshape_3[0][0],  │
│ (MultiHeadAttentio… │                   │            │ reshape_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 1280)      │          0 │ multi_head_atten… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_10          │ (None, 1280)      │          0 │ global_average_p… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 128)       │    163,968 │ dropout_10[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_11          │ (None, 128)       │          0 │ dense_6[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 13)        │      1,677 │ dropout_11[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,047,885 (19.26 MB)

 Trainable params: 2,789,901 (10.64 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [38]:
import tensorflow as tf

try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
    print("✅ Running on TPU")
except:
    strategy = tf.distribute.get_strategy()
    print("⚠️ TPU not found, running on default strategy (CPU/GPU)")

⚠️ TPU not found, running on default strategy (CPU/GPU)


In [39]:
with strategy.scope():
    loss_fn = focal_loss_with_class_weights(class_weights, gamma=2.0, alpha=0.25)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss=loss_fn,
        metrics=[tf.keras.metrics.AUC(curve='PR', multi_label=True, name='pr_auc')]
    )

In [40]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=6,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5)
    ]
)

Epoch 1/6
115/115 ━━━━━━━━━━━━━━━━━━━━ 145s 1s/step - loss: 0.0432 - pr_auc: 0.1209 - val_loss: 0.0180 - val_pr_auc: 0.6064 - learning_rate: 1.0000e-04
Epoch 2/6
115/115 ━━━━━━━━━━━━━━━━━━━━ 135s 1s/step - loss: 0.0217 - pr_auc: 0.4557 - val_loss: 0.0137 - val_pr_auc: 0.7412 - learning_rate: 1.0000e-04
Epoch 3/6
115/115 ━━━━━━━━━━━━━━━━━━━━ 129s 1s/step - loss: 0.0155 - pr_auc: 0.6624 - val_loss: 0.0103 - val_pr_auc: 0.8331 - learning_rate: 1.0000e-04
Epoch 4/6
115/115 ━━━━━━━━━━━━━━━━━━━━ 135s 1s/step - loss: 0.0112 - pr_auc: 0.7938 - val_loss: 0.0097 - val_pr_auc: 0.8613 - learning_rate: 1.0000e-04
Epoch 5/6
115/115 ━━━━━━━━━━━━━━━━━━━━ 132s 1s/step - loss: 0.0083 - pr_auc: 0.8759 - val_loss: 0.0081 - val_pr_auc: 0.8884 - learning_rate: 1.0000e-04
Epoch 6/6
115/115 ━━━━━━━━━━━━━━━━━━━━ 136s 1s/step - loss: 0.0063 - pr_auc: 0.9259 - val_loss: 0.0078 - val_pr_auc: 0.9036 - learning_rate: 1.0000e-04


In [41]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=4,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5)
    ]
)

Epoch 1/4
115/115 ━━━━━━━━━━━━━━━━━━━━ 150s 1s/step - loss: 0.0048 - pr_auc: 0.9512 - val_loss: 0.0080 - val_pr_auc: 0.9149 - learning_rate: 1.0000e-04
Epoch 2/4
115/115 ━━━━━━━━━━━━━━━━━━━━ 192s 1s/step - loss: 0.0034 - pr_auc: 0.9730 - val_loss: 0.0077 - val_pr_auc: 0.9137 - learning_rate: 1.0000e-04
Epoch 3/4
115/115 ━━━━━━━━━━━━━━━━━━━━ 133s 1s/step - loss: 0.0030 - pr_auc: 0.9804 - val_loss: 0.0082 - val_pr_auc: 0.9138 - learning_rate: 1.0000e-04
Epoch 4/4
115/115 ━━━━━━━━━━━━━━━━━━━━ 141s 1s/step - loss: 0.0022 - pr_auc: 0.9883 - val_loss: 0.0088 - val_pr_auc: 0.9029 - learning_rate: 1.0000e-04


In [42]:
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score, average_precision_score, roc_auc_score
from sklearn.metrics import jaccard_score, hamming_loss

# Collect true labels and predictions from validation/test set
y_true = np.concatenate([y for _, y in test_ds], axis=0)   # or test_ds
y_probs = np.concatenate([model.predict(x) for x, _ in test_ds], axis=0)
y_pred = (y_probs >= 0.5).astype(int)

# ----------------------------
# Micro metrics (global)
# ----------------------------
precision_micro = precision_score(y_true, y_pred, average='micro', zero_division=0)
recall_micro = recall_score(y_true, y_pred, average='micro', zero_division=0)
f1_micro = f1_score(y_true, y_pred, average='micro', zero_division=0)

# ----------------------------
# Macro metrics (average per class)
# ----------------------------
precision_macro = precision_score(y_true, y_pred, average='macro', zero_division=0)
recall_macro = recall_score(y_true, y_pred, average='macro', zero_division=0)
f1_macro = f1_score(y_true, y_pred, average='macro', zero_division=0)

# ----------------------------
# Per-class metrics
# ----------------------------
num_classes = y_true.shape[1]
avg_precision = [average_precision_score(y_true[:,i], y_probs[:,i]) for i in range(num_classes)]
roc_auc = [roc_auc_score(y_true[:,i], y_probs[:,i]) for i in range(num_classes)]

# ----------------------------
# Multi-label specific metrics
# ----------------------------
jaccard_micro = jaccard_score(y_true, y_pred, average='micro')
jaccard_macro = jaccard_score(y_true, y_pred, average='macro')
hamming = hamming_loss(y_true, y_pred)

# ----------------------------
# Print results
# ----------------------------
print("\n=== Multi-label metrics ===")
print(f"Precision micro: {precision_micro:.4f}, Recall micro: {recall_micro:.4f}, F1 micro: {f1_micro:.4f}")
print(f"Precision macro: {precision_macro:.4f}, Recall macro: {recall_macro:.4f}, F1 macro: {f1_macro:.4f}")
print(f"Jaccard index (micro): {jaccard_micro:.4f}, Jaccard index (macro): {jaccard_macro:.4f}")
print(f"Hamming loss: {hamming:.4f}")
print("Average precision per class:", np.round(avg_precision, 3))
print("ROC-AUC per class:", np.round(roc_auc, 3))

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 657ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 653ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 667ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 666ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 643ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 666ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 681ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 651ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 666ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 655ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 654ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 642ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 667ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 665ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 645ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 659ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 656ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 914ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step

=== Multi-label metrics ===
Precision micro: 

In [43]:
model.save('target_recognition_model_pilot.keras')

Trying few more epochs and see

In [44]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=2,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5)
    ]
)

Epoch 1/2
115/115 ━━━━━━━━━━━━━━━━━━━━ 136s 1s/step - loss: 0.0028 - pr_auc: 0.9849 - val_loss: 0.0091 - val_pr_auc: 0.9122 - learning_rate: 5.0000e-05
Epoch 2/2
115/115 ━━━━━━━━━━━━━━━━━━━━ 138s 1s/step - loss: 0.0019 - pr_auc: 0.9850 - val_loss: 0.0081 - val_pr_auc: 0.9179 - learning_rate: 5.0000e-05


In [45]:
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score, average_precision_score, roc_auc_score
from sklearn.metrics import jaccard_score, hamming_loss

# Collect true labels and predictions from validation/test set
y_true = np.concatenate([y for _, y in test_ds], axis=0)   # or test_ds
y_probs = np.concatenate([model.predict(x) for x, _ in test_ds], axis=0)
y_pred = (y_probs >= 0.5).astype(int)

# ----------------------------
# Micro metrics (global)
# ----------------------------
precision_micro = precision_score(y_true, y_pred, average='micro', zero_division=0)
recall_micro = recall_score(y_true, y_pred, average='micro', zero_division=0)
f1_micro = f1_score(y_true, y_pred, average='micro', zero_division=0)

# ----------------------------
# Macro metrics (average per class)
# ----------------------------
precision_macro = precision_score(y_true, y_pred, average='macro', zero_division=0)
recall_macro = recall_score(y_true, y_pred, average='macro', zero_division=0)
f1_macro = f1_score(y_true, y_pred, average='macro', zero_division=0)

# ----------------------------
# Per-class metrics
# ----------------------------
num_classes = y_true.shape[1]
avg_precision = [average_precision_score(y_true[:,i], y_probs[:,i]) for i in range(num_classes)]
roc_auc = [roc_auc_score(y_true[:,i], y_probs[:,i]) for i in range(num_classes)]

# ----------------------------
# Multi-label specific metrics
# ----------------------------
jaccard_micro = jaccard_score(y_true, y_pred, average='micro')
jaccard_macro = jaccard_score(y_true, y_pred, average='macro')
hamming = hamming_loss(y_true, y_pred)

# ----------------------------
# Print results
# ----------------------------
print("\n=== Multi-label metrics ===")
print(f"Precision micro: {precision_micro:.4f}, Recall micro: {recall_micro:.4f}, F1 micro: {f1_micro:.4f}")
print(f"Precision macro: {precision_macro:.4f}, Recall macro: {recall_macro:.4f}, F1 macro: {f1_macro:.4f}")
print(f"Jaccard index (micro): {jaccard_micro:.4f}, Jaccard index (macro): {jaccard_macro:.4f}")
print(f"Hamming loss: {hamming:.4f}")
print("Average precision per class:", np.round(avg_precision, 3))
print("ROC-AUC per class:", np.round(roc_auc, 3))

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 686ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 727ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 728ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 677ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 723ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 690ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 690ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 700ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 940ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 766ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 665ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 729ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 677ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 687ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 701ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 690ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 695ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 678ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 689ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 379ms/step

=== Multi-label metrics ===
Precision m

In [46]:
model.save('target_recognition_model.keras')